In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 1000)
aspect_name = 'access'
revised = 'revised'

In [2]:
result_path = 'batch1/batch1_1000_result/formatted_result/Batch_1000_' + aspect_name + '_'+ revised +'_three_workers_formatted_result.csv'
df = pd.read_csv(result_path)
A_ROW = []
for index, answerAspect in enumerate(df['subaspect']):
    formatted_aspect = [int(each.strip(' ')) if each != '' else 0 for each in answerAspect[1:-1].split(',') ]
    formatted_groundaspect = [int(each.strip(' ')) if each != '' else 0 for each in df.iloc[index]['ground_truth_subaspect'][1:-1].split(',') ]
    if set(formatted_aspect).intersection(formatted_groundaspect) != set():
        A_ROW.append('right')
    else:
        A_ROW.append('wrong')
df['SubAspect_RightOrWrong'] = A_ROW

S_ROW = []
for index, answerAspect in enumerate(df['sentiment']):
    if answerAspect == df.iloc[index]['ground_truth_sentiment']:
        S_ROW.append('right')
    else:
        S_ROW.append('wrong')
df['Sentiment_RightOrWrong'] = S_ROW

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4420 entries, 0 to 4419
Data columns (total 22 columns):
HITId                     4420 non-null object
HITTypeId                 4420 non-null object
AssignmentId              4420 non-null object
WorkerId                  4420 non-null object
AssignmentStatus          4420 non-null object
WorkTimeInSeconds         4420 non-null int64
LifetimeApprovalRate      4420 non-null object
Last30DaysApprovalRate    4420 non-null object
aspect                    4420 non-null object
course_of_problem         4420 non-null object
group                     4420 non-null object
id                        4420 non-null object
text                      4420 non-null object
trainOrtest               4420 non-null object
batch                     4420 non-null int64
treatment                 4420 non-null object
ground_truth_subaspect    4420 non-null object
ground_truth_sentiment    4420 non-null int64
subaspect                 4420 non-null object
sen

# Evaluation on Whole Labeling

In [3]:
from collections import Counter
import math
not_about_dic = {"delays": 5, "costs": 5, "access": 5, "errors": 5, "trusts": 3, "treatments":3}
not_about = not_about_dic[aspect_name]

def sentiment_vote(x, columns):
    new_columns = [each for each in columns if each.startswith('worker-')] 
    num_workers = 0
    sentiment_score = 0
    cnt = Counter()
    for worker in new_columns:
        num_workers += 1
        sentiment_score += int(x[worker])
        cnt[x[worker]] += 1
#     if 1.0*sentiment_score / num_workers == 1.4 and cnt[0] == 0:
#         return 2
    return round(sentiment_score / num_workers)

def subaspect_vote(x, columns):
    new_columns = [each for each in columns if each.startswith('worker-')] 
    num = len(new_columns)
    
    ct = Counter()
    for worker in new_columns:
        for each_ans in x[worker][1:-1].split(','):
            ct[each_ans.strip()] += 1 
#     result = [k for k, v in ct.items() if v >= math.ceil(num / 2)]
    result = [k for k, v in ct.items() if v >= math.floor(num / 2)]
    if len(result) == 0 and '0' not in ct.keys():
        result = [not_about]
        return result
    if len(result) == 0 and '0' in ct.keys():
        result = ['0']
        return result
    return result

In [4]:
# Transform format to show workers answers by each text
temp = df[['batch', 'id']]
temp_df = df[['subaspect','id']]
temp_df['worker_row'] = temp.groupby(['id'])['batch'].rank(method='first').apply(lambda x: 'worker-' + str(int(x)))
formated_subaspect_answer = temp_df.pivot(index='id', columns='worker_row', values='subaspect')

subaspect_compare = df[['id', 'text','ground_truth_subaspect']].drop_duplicates().join(formated_subaspect_answer, on='id').reset_index(drop=True)
subaspect_compare['voting'] = subaspect_compare.apply(lambda x: subaspect_vote(x, subaspect_compare.columns), axis=1)
subaspect_compare['rightOrWrong'] = subaspect_compare.apply(lambda x: True if len(set(x['ground_truth_subaspect'][1:-1].split(",")).intersection(set(x['voting'])))>0 else False, axis=1)
subaspect_compare_non_zero = subaspect_compare[subaspect_compare.ground_truth_subaspect != '[0]']
subaspect_compare_non_zero['voting'] = subaspect_compare_non_zero.apply(lambda x: subaspect_vote(x, subaspect_compare_non_zero.columns), axis=1)
subaspect_compare_non_zero['rightOrWrong'] = subaspect_compare_non_zero.apply(lambda x: True if len(set(x['ground_truth_subaspect'][1:-1].split(",")).intersection(set(x['voting'])))>0 else False, axis=1)
print("================================================================================================")
print("The accuracy of voted worker's answers to the subaspect is [Within Aspects]: ",round(sum(subaspect_compare_non_zero['rightOrWrong'] / 1.0) / subaspect_compare_non_zero.shape[0], 3))
print("================================================================================================")
non_zero_temp = subaspect_compare_non_zero[['text', 'ground_truth_subaspect', 'worker-1', 'worker-2', 'worker-3',  'worker-4',  'worker-5', 'voting',  'rightOrWrong']]
non_zero_false = non_zero_temp[non_zero_temp.rightOrWrong == False]
# for each in non_zero_false.iterrows():
#     print(each[1]['text'])
#     print('worker1: ', each[1]['worker-1'])
#     print('worker2: ',each[1]['worker-2'])
#     print('worker3: ',each[1]['worker-3'])
#     print('worker4: ',each[1]['worker-4'])
#     print('worker5: ',each[1]['worker-5'])
#     print('Vote: ',each[1]['voting'])
#     print('groud truth:', each[1]['ground_truth_subaspect'])

print("================================================================================================")
print("The accuracy of voted worker's answers to the subaspect is: [Total records]",round(sum(subaspect_compare['rightOrWrong'] / 1.0) / subaspect_compare.shape[0], 3))
print("================================================================================================")
aspects_temp = subaspect_compare[[ 'text','ground_truth_subaspect', 'worker-1', 'worker-2', 'worker-3', 'worker-4',  'worker-5', 'voting',  'rightOrWrong']]
aspects_temp_false = aspects_temp [aspects_temp.rightOrWrong == False]
# for each in aspects_temp_false.iterrows():
#     print(each[1]['text'])
#     print('worker1: ', each[1]['worker-1'])
#     print('worker2: ',each[1]['worker-2'])
#     print('worker3: ',each[1]['worker-3'])
#     print('worker4: ',each[1]['worker-4'])
#     print('worker5: ',each[1]['worker-5'])
#     print('Vote: ',each[1]['voting'])
#     print('groud truth:', each[1]['ground_truth_subaspect'])

/Users/jaden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


TypeError: ("'float' object is not subscriptable", 'occurred at index 0')

In [647]:
# [0] Not about Delays (You should not choose any option below if you check this option)
# [1] Delays in Appointments and Meetings with People in healthcare (Doctors, Consultants, Therapists, Dentists, Prescribing etc.) 
# [2] Delays in Scheduling, Conducting, or Receiving Results from Medical Tests and Scans
# [3] Delays in Getting or Receiving Treatments (Medical Procedures like Surgery, Therapies, Ambulance service, etc.)
# [4] Delays in Recovery after Receiving Treatment
# [5] Delays in something else that is ambiguous or not related to any option above

# print(subaspect_compare_non_zero.loc[44]['text'])
# print(subaspect_compare_non_zero.loc[44]['ground_truth_subaspect'])
# print(subaspect_compare_non_zero.loc[44]['worker-1'])
# print(subaspect_compare_non_zero.loc[44]['worker-2'])
# print(subaspect_compare_non_zero.loc[44]['worker-3'])
# print(subaspect_compare_non_zero.loc[44]['voting'])

In [736]:
# Transform format to show workers answers by each text
temp = df[['batch', 'id']]
temp_df = df[['sentiment','id']]
temp_df['worker_row'] = temp.groupby(['id'])['batch'].rank(method='first').apply(lambda x: 'worker-' + str(int(x)))
formated_sentiment_answer = temp_df.pivot(index='id', columns='worker_row', values='sentiment')


sentiment_compare = df[['id', 'text','ground_truth_sentiment']].drop_duplicates().join(formated_sentiment_answer, on='id').reset_index(drop=True)
sentiment_compare['voting'] = sentiment_compare.apply(lambda x: sentiment_vote(x, sentiment_compare.columns), axis=1)
sentiment_compare['rightOrWrong'] = sentiment_compare['ground_truth_sentiment'] == sentiment_compare['voting']
print("===================================================================")
print("The accuracy of voted worker's answers to the sentiment is: ",sum(sentiment_compare['rightOrWrong'] / 1.0) / sentiment_compare.shape[0])
print("===================================================================")

sentiment_temp = sentiment_compare[['text', 'ground_truth_sentiment', 'worker-1', 'worker-2', 'worker-3',  'worker-4',  'worker-5', 'voting',  'rightOrWrong']]
sentiment_false = sentiment_temp[sentiment_temp.rightOrWrong == False]
# for each in sentiment_false.iterrows():
#     print(each[1]['text'])
#     print('worker1: ', each[1]['worker-1'])
#     print('worker2: ',each[1]['worker-2'])
#     print('worker3: ',each[1]['worker-3'])
#     print('worker4: ',each[1]['worker-4'])
#     print('worker5: ',each[1]['worker-5'])
#     print('Vote: ',each[1]['voting'])
#     print('groud truth:', each[1]['ground_truth_sentiment'])
    
sentiment_compare = sentiment_compare[sentiment_compare.ground_truth_sentiment!=0]
sentiment_compare['voting'] = sentiment_compare.apply(lambda x: sentiment_vote(x, sentiment_compare.columns), axis=1)
sentiment_compare['rightOrWrong'] = sentiment_compare['ground_truth_sentiment'] == sentiment_compare['voting']
print("===================================================================")
print("The accuracy of voted worker's answers to the sentiment is (Negative): ",sum(sentiment_compare['rightOrWrong'] / 1.0) / sentiment_compare.shape[0])
print("===================================================================")

non_neutral_temp = sentiment_compare[['text', 'ground_truth_sentiment', 'worker-1', 'worker-2', 'worker-3',  'worker-4',  'worker-5', 'voting',  'rightOrWrong']]
non_neutral_false = non_neutral_temp[non_neutral_temp.rightOrWrong == False]
# for each in non_neutral_false.iterrows():
#     print(each[1]['text'])
#     print('worker1: ', each[1]['worker-1'])
#     print('worker2: ',each[1]['worker-2'])
#     print('worker3: ',each[1]['worker-3'])
#     print('worker4: ',each[1]['worker-4'])
#     print('worker5: ',each[1]['worker-5'])
#     print('Vote: ',each[1]['voting'])
#     print('groud truth:', each[1]['ground_truth_sentiment'])

The accuracy of voted worker's answers to the sentiment is:  0.657
The accuracy of voted worker's answers to the sentiment is (Negative):  0.5572519083969466


/Users/jaden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [649]:
# n = 22
# print(sentiment_compare.loc[n]['text'])
# print(sentiment_compare.loc[n]['ground_truth_sentiment'])
# print(sentiment_compare.loc[n]['worker-1'])
# print(sentiment_compare.loc[n]['worker-2'])
# print(sentiment_compare.loc[n]['worker-3'])
# print(sentiment_compare.loc[n]['voting'])

# Evaluation on Workers

In [650]:
numHit_MIN = 20

In [ ]:
# used for evaluation of workers in the case of no ground truth labeled on our own
# df.drop(['ground_truth_subaspect'])
# df.join(subaspect_compare[['id', 'voting']], on='id')
# df.rename(columns = {'voting':'ground_truth_subaspect'}, inplace = True)
# df.drop(['ground_truth_sentiment'])
# df.join(sentiment_compare[['id', 'voting']], on='id')
# df.rename(columns = {'voting':'ground_truth_sentiment'}, inplace = True)

## Evaluation of wokers performance on aspect question

In [651]:
temp_df = df[['WorkerId', 'AssignmentId', 'SubAspect_RightOrWrong']].groupby(['WorkerId', 'SubAspect_RightOrWrong']).count().reset_index()
eval_aspect_df = temp_df.pivot(index='WorkerId', columns='SubAspect_RightOrWrong', values='AssignmentId')
eval_aspect_df['total'] = eval_aspect_df['right'] + eval_aspect_df['wrong']
eval_aspect_df['accuracy'] = eval_aspect_df['right'] / eval_aspect_df['total']
condition_eval_aspect_df = eval_aspect_df.sort_values('accuracy', ascending=False)

worker_good = condition_eval_aspect_df[condition_eval_aspect_df.accuracy >= 0.60]
print("good workers: \n", worker_good)

worker_pay = condition_eval_aspect_df[condition_eval_aspect_df.accuracy >= 0.4]
print("to pay workers: \n",worker_pay)


# condition_eval_aspect_df[condition_eval_aspect_df.total >= numHit_MIN]
worker_pay

good workers: 
 SubAspect_RightOrWrong  right  wrong  total  accuracy
WorkerId                                             
A2WYGPR0ECWJFZ            6.0    4.0   10.0       0.6
A28QOYIO3C8WFY            6.0    4.0   10.0       0.6
to pay workers: 
 SubAspect_RightOrWrong  right  wrong  total  accuracy
WorkerId                                             
A2WYGPR0ECWJFZ            6.0    4.0   10.0  0.600000
A28QOYIO3C8WFY            6.0    4.0   10.0  0.600000
A19Q676CHKCVOR           35.0   25.0   60.0  0.583333
A31BA1WLSWUSY1           17.0   13.0   30.0  0.566667
A304S6RPVBSZRV          138.0  122.0  260.0  0.530769
AWVJ2USKEKN1P            95.0   85.0  180.0  0.527778
A16SAB417Y96G4            5.0    5.0   10.0  0.500000
A3QYVLTAIHB6OI          124.0  126.0  250.0  0.496000
ATEWATSZJIDBW           110.0  120.0  230.0  0.478261
A2HITRHQ441VHA           37.0   43.0   80.0  0.462500
A35Z81OG580PL8           36.0   44.0   80.0  0.450000
AUZBIQJ0A5FM9            61.0   89.0  150.0  0.4

SubAspect_RightOrWrong,right,wrong,total,accuracy
WorkerId,,,,
A2WYGPR0ECWJFZ,6.0,4.0,10.0,0.600000
A28QOYIO3C8WFY,6.0,4.0,10.0,0.600000
A19Q676CHKCVOR,35.0,25.0,60.0,0.583333
A31BA1WLSWUSY1,17.0,13.0,30.0,0.566667
A304S6RPVBSZRV,138.0,122.0,260.0,0.530769
AWVJ2USKEKN1P,95.0,85.0,180.0,0.527778
A16SAB417Y96G4,5.0,5.0,10.0,0.500000
A3QYVLTAIHB6OI,124.0,126.0,250.0,0.496000
ATEWATSZJIDBW,110.0,120.0,230.0,0.478261


## Evaluation of wokers performance on aspect question (exclude "Not about")

In [652]:
filter_aspect_df = df[df.ground_truth_subaspect != '[0]']
temp_df = filter_aspect_df[['WorkerId', 'AssignmentId', 'SubAspect_RightOrWrong']].groupby(['WorkerId', 'SubAspect_RightOrWrong']).count().reset_index()
eval_aspect_df = temp_df.pivot(index='WorkerId', columns='SubAspect_RightOrWrong', values='AssignmentId')
eval_aspect_df['total'] = eval_aspect_df['right'] + eval_aspect_df['wrong']
eval_aspect_df['accuracy'] = eval_aspect_df['right'] / eval_aspect_df['total']
condition_eval_aspect_df = eval_aspect_df.sort_values('accuracy', ascending=False)
condition_eval_aspect_df[condition_eval_aspect_df.total > numHit_MIN]

SubAspect_RightOrWrong,right,wrong,total,accuracy
WorkerId,,,,
A54HGV3WF2822,29.0,25.0,54.0,0.537037
A2MYK33JT321O6,38.0,36.0,74.0,0.513514
A2P7IU3JLL4VW2,34.0,42.0,76.0,0.447368
A37V7156T1VXI7,35.0,54.0,89.0,0.393258
A10CWGO4F50WBY,9.0,14.0,23.0,0.391304
A3AJAE98ONTPBK,24.0,42.0,66.0,0.363636
A2OOUWCIBOZAK4,19.0,48.0,67.0,0.283582
A32OMKDQ4FDXW8,17.0,44.0,61.0,0.278689
A2QU0ASAUX4S64,16.0,42.0,58.0,0.275862


## Evaluation of wokers performance on sentiment question

In [653]:
temp_df = df[['WorkerId', 'AssignmentId', 'Sentiment_RightOrWrong']].groupby(['WorkerId', 'Sentiment_RightOrWrong']).count().reset_index()
eval_sentiment_df = temp_df.pivot(index='WorkerId', columns='Sentiment_RightOrWrong', values='AssignmentId')
eval_sentiment_df['total'] = eval_sentiment_df['right'] + eval_sentiment_df['wrong']
eval_sentiment_df['accuracy'] = eval_sentiment_df['right'] / eval_sentiment_df['total']
condition_eval_sentiment_df = eval_sentiment_df.sort_values('accuracy', ascending=False)
condition_eval_sentiment_df[condition_eval_sentiment_df.total > numHit_MIN]

Sentiment_RightOrWrong,right,wrong,total,accuracy
WorkerId,,,,
A2HITRHQ441VHA,50,30,80,0.625000
A31BA1WLSWUSY1,18,12,30,0.600000
A304S6RPVBSZRV,152,108,260,0.584615
A19Q676CHKCVOR,35,25,60,0.583333
A1JWDEOZ639RU,156,114,270,0.577778
AO2WNSGOXAX52,86,64,150,0.573333
A2OOUWCIBOZAK4,138,112,250,0.552000
AWVJ2USKEKN1P,97,83,180,0.538889
ACRPGQAXICOOT,50,50,100,0.500000


## Evaluation of workers on sentiment question (exclude non-negative)

In [654]:
filter_sentiment_df = df[df.ground_truth_sentiment != 0]
temp_df = filter_sentiment_df[['WorkerId', 'AssignmentId', 'Sentiment_RightOrWrong']].groupby(['WorkerId', 'Sentiment_RightOrWrong']).count().reset_index()
eval_sentiment_df = temp_df.pivot(index='WorkerId', columns='Sentiment_RightOrWrong', values='AssignmentId')
eval_sentiment_df['total'] = eval_sentiment_df['right'] + eval_sentiment_df['wrong']
eval_sentiment_df['accuracy'] = eval_sentiment_df['right'] / eval_sentiment_df['total']
condition_eval_sentiment_df = eval_sentiment_df.sort_values('accuracy', ascending=False)

worker = condition_eval_aspect_df[condition_eval_aspect_df.accuracy <= 0.3]
print(worker.index)
condition_eval_sentiment_df[condition_eval_sentiment_df.total > numHit_MIN]

Index(['A2OOUWCIBOZAK4', 'A32OMKDQ4FDXW8', 'A2QU0ASAUX4S64', 'A1ZH4F9NSH0Z7G', 'A16SAB417Y96G4', 'ACRPGQAXICOOT', 'A1JWDEOZ639RU', 'A315BL34XME4SS', 'A1LD1TBFE4ELEA', 'A3QYVLTAIHB6OI', 'A2B6OA4V8YABCZ', 'AUZBIQJ0A5FM9', 'A2HITRHQ441VHA', 'A80ZQS34472ZL', 'A23BHIE4PDKP1Y', 'A304S6RPVBSZRV', 'A3LPEOCWBXIVYB', 'AO2WNSGOXAX52', 'AWVJ2USKEKN1P', 'ATEWATSZJIDBW', 'A35Z81OG580PL8'], dtype='object', name='WorkerId')


Sentiment_RightOrWrong,right,wrong,total,accuracy
WorkerId,,,,
A2P7IU3JLL4VW2,83.0,15.0,98.0,0.846939
A3AJAE98ONTPBK,71.0,19.0,90.0,0.788889
A2QU0ASAUX4S64,56.0,15.0,71.0,0.788732
A37V7156T1VXI7,85.0,24.0,109.0,0.779817
A2MYK33JT321O6,80.0,24.0,104.0,0.769231
A54HGV3WF2822,57.0,20.0,77.0,0.740260
A32OMKDQ4FDXW8,65.0,23.0,88.0,0.738636
A3LPEOCWBXIVYB,13.0,17.0,30.0,0.433333
ATEWATSZJIDBW,37.0,61.0,98.0,0.377551
